# Libraries

In [ ]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
patent_nb = pd.read_csv("./Data/patents_inproducts.csv")
list_patents = patent_nb.patent_number.values

In [ ]:
df = pd.DataFrame()
for index,i in enumerate(list_patents):
    # extract url of game page
    token1 = "https://api.patentsview.org/patents/query?q={%22patent_number%22:"
    token2 = "}&f=[%22patent_number%22,%22patent_title%22,%22patent_date%22,%22patent_kind%22,%22patent_type%22]&s=[{%22patent_title%22:%22asc%22}]"
    url = token1 + str(i) + token2
    r = requests.get(url)
    if index%100==0:
        print(index)
    if r.status_code == 200:
        html = r.text
    else:
        print("error : page not found")

    # initialize game line
    patent = {"patent_number": i, "patent_title": None,
            "patent_date": None, "patent_kind": None, 
            "patent_type": None}

    # extract data from page
    bs = BeautifulSoup(html, 'html.parser')
    if bs.contents[0].find('total_patent_count":0') <0:
        Dict = ast.literal_eval(html)["patents"][0]
        patent["patent_title"] = Dict["patent_title"]
        patent["patent_date"]=Dict["patent_date"]
        patent["patent_kind"]=Dict["patent_kind"]
        patent["patent_type"]=Dict["patent_type"]
    #bs.find('patent_name', class_="details_block").findAll('b')[0].next_sibling
        df = df.append(patent, ignore_index=True)
print(df)
df.to_csv(index=False, header=True)